In [1]:
import pandas as pd
from mip import Model, MAXIMIZE, CBC, BINARY, OptimizationStatus, xsum

frame = pd.read_csv('table_for_optimization.csv')
frame.head()

,client_id,product,channel,score
0,1,product1,call,0.056751
1,1,product2,call,0.196437
2,1,product3,call,0.263622
3,1,product1,sms,0.036902
4,1,product2,sms,0.055962


In [2]:
def optimize(frame: pd.DataFrame, channel_limits: dict) -> list:
    """
    Возвращает массив оптимальных предложений
    """
    
    df = frame.copy()
    
    #создание модели
    model = Model(sense=MAXIMIZE, solver_name=CBC)

    #вектор бинарных переменных задачи
    x = [model.add_var(var_type=BINARY) for i in range(df.shape[0])]
    df['x'] = x

    #целевая функция
    #функция xsum значительно ускоряет суммирование https://docs.python-mip.com/en/latest/classes.html#mip.xsum    
    model.objective = xsum(df['score'] * df['x'])

    #ограничения на количество коммуникаций в каждом канале
    for channel in df.channel.unique():
        model += (xsum(df[df.channel==channel]['x']) <= channel_limits[channel])

    #ограничения на количество продуктов для каждого клиента (не более одного продукта на клиента)
    for product_sum in df.groupby(['client_id'])['x'].apply(xsum):
        model += (product_sum <= 1)
        
    status = model.optimize(max_seconds=300)
    
    del df
    
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        return [var.x for var in model.vars]
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('No feasible solution found')

In [3]:
#объем доступных коммуникаций в каналах
CHANNELS_LIMITS = {
    'call': 200,
    'sms': 500
}

optimal_decisions = optimize(frame=frame, channel_limits=CHANNELS_LIMITS)
frame['optimal_decision'] = optimal_decisions

#распределение продуктов в каналах
frame[frame['optimal_decision']==1].groupby(['channel', 'product']).\
                                    agg({'client_id': 'count'}).\
                                    rename(columns={'client_id': 'client_cnt'})

client_cnt
channel product             
call    product1          62
        product2          61
        product3          77
sms     product1         171
        product2         155
        product3         174